In [1]:
import numpy as np
import torch

In [2]:
# Función para padding con ceros

def zero_padding(X, pad):
  """
  Argumentos:
    X: Array numpy de entrada con dimensiones (batch_size, n_C, n_H, n_W)
    pad: Entero representando la cantidad de filas y columnas que se deben agregar con ceros

  Retorna:
    X_padded: Array numpy con dimensiones (batch_size, n_C, n_H + 2*pad, n_W + 2*pad)
  """

  X_padded = np.pad(X, ((0,0), (0,0), (pad, pad), (pad, pad)), mode='constant', constant_values = (0,0)) #Tiene (0,0),(0,0) porque n_C seguramente = 2 (número de capas=2) 

  return X_padded

In [3]:
# Función para realizar la operación de convolución

def convolve(X, W, b):
  """
  Argumentos:
    X: Array numpy de entrada con dimensiones (filter_size, filter_size, n_C_prev)
    W: Array numpy con los pesos de un filtro con dimensiones (filter_size, filter_size, n_C_prev)
    b: Entero con el valor de bias de la capa actual

    IMPORTANTE: EN ESTE CASO, TANTO EL FILTRO COMO EL INPUT TIENEN LA MISMA DIMENSIÓN 
  Retorna:
    Z: Entero con el valor del resultado
  """

  # Multiplico elemento a elemento el valor de entrada con los pesos del filtro
  aux = X * W
  # Realizo la suma de todos los elementos
  aux = np.sum(aux)
  # Le sumo el valor del bias para obtener Z
  Z = aux + float(b)

  return Z

In [4]:
# Función para realizar el forward pass de una capa convolucional

def conv_forward(layer_input, W, b, stride, padding):
  """
  Argumentos:
    layer_input: Array numpy con los valores de entrada a la capa convolucional (batch_size, n_C_prev, n_H_prev, n_W_prev)
    W: Array numpy con los pesos de los filtros utilizados en la capa actual (n_C, n_C_prev, filter_size, filter_size)
    b: Array numpy con los valores de bias utilizados en la capa actual (1, 1, 1, n_C)
    stride: Entero con el valor de stride utilizado en la capa actual.
    padding: 

  Retorna:
    Z: Array numpy con los valores de salida de la capa convolucional (batch_size, n_H, n_W, n_C)
  """

  # Obtengo las dimensiones de la entrada
  (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape

  # Obtengo las dimensiones de los filtros
  (n_C, n_C_prev, filter_size, filter_size) = W.shape

  # Calculo las dimensiones del volumen de salida de la capa actual
  n_H = int((n_H_prev + 2*padding - filter_size)/stride + 1)
  n_W = int((n_W_prev + 2*padding - filter_size)/stride + 1)

  # Inicializo el volumen de salida con ceros
  Z = np.zeros([batch_size, n_C, n_H, n_W])

  # Agrego padding con ceros al volumen de entrada
  layer_input_padded = zero_padding(layer_input, padding)

  # Comienzo iterando sobre cada ejemplo del batch
  for i in range(batch_size):

    # Itero sobre el eje vertical del volumen de salida
    for h in range(n_H):
      # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
      y_start = stride * h
      y_end = y_start + filter_size

      # Itero sobre el eje horizontal del volumen de salida
      for w in range(n_W):
        # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        x_start = stride * w
        x_end = x_start + filter_size

        # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
        slice_from_input_padded = layer_input_padded[i, :, y_start:y_end, x_start:x_end]
        
        # Itero sobre la cantidad de canales del volumen de salida
        for c in range(n_C):

          # Obtengo el valor del filtro y bias del canal correspondiente
          filter = W[c, :, :, :]
          bias = b[c]

          # Computo la operación de convolución para esta ventana
          Z[i, c, h, w] = convolve(slice_from_input_padded, filter, bias)
  
  return Z

Ejecución

In [5]:
# Dimensiones de la entrada
batch_size = 10
input_height, input_width = (7, 7)
input_channels = 4

# Dimensiones de la convolucional
filters = 8
filter_size = 3
stride = 2
pad = 1

# Variables de prueba
test_array = np.random.randn(batch_size, input_channels, input_height, input_width)
W = np.random.randn(filters, input_channels, filter_size, filter_size)
b = np.random.randn(filters)


conv_result = conv_forward(test_array, W, b, stride, pad)
conv_result_pyt = torch.nn.functional.conv2d(torch.tensor(test_array), torch.tensor(W), torch.tensor(b), stride, pad)

assert(conv_result.shape == conv_result_pyt.shape)
print("Convolución: Result shape: {}".format(conv_result.shape))
print("Convolución: Result value: {}".format(conv_result[1, 1, 1, 1]))
print("Convolución: Result shape: {}".format(conv_result_pyt.shape))
print("Convolución: Result value: {}".format(conv_result_pyt[1, 1, 1, 1]))

Convolución: Result shape: (10, 8, 4, 4)
Convolución: Result value: 3.2302593979703746
Convolución: Result shape: torch.Size([10, 8, 4, 4])
Convolución: Result value: 3.2302593979703738


Con el código utilizado en clase, implementar una clase de Python similar a esta. Se le deben pasar como argumentos:
    
    Cantidad de filtros
    Tamaño de los filtros
    Stride
    Tipo de padding (“valid” o “same”)
Debe tener un método “forward”, al cual se le pasa la matriz de entrada y devuelve la salida.
Y un método “backward”, al cual se le pasa una matriz con ∂L/∂O y devuelve dos matrices, una con ∂L/∂F y otra con ∂L/∂X.


In [6]:
import numpy as np
import torch

In [8]:
# Función para padding con ceros

def zero_padding(X, padding_mode, F):
  """
  Argumentos:
    X: Array numpy de entrada con dimensiones (batch_size, n_C, n_H, n_W)
    padding_mode: {'valid' , 'same'}. Valid = no padding. same --> output.shape = input.shape
  Retorna:
    X_padded: Array numpy con dimensiones (batch_size, n_C, n_H + 2*pad, n_W + 2*pad)
  """ 
  # n - f + 2p + 1 = n
  # p = (f-1)/2
  pad_h = np.divide((F.shape[1]-int(1)),2)
  pad_w = np.divide((F.shape[2]-int(1)),2)
  if padding_mode == 'same':
    
    X_padded = np.pad(X, ((0,0), (0,0), (pad_h, pad_w), (pad_h, pad_w)), mode='constant', constant_values = (0,0)) #Tiene (0,0),(0,0) porque n_C seguramente = 2 (número de capas=2) 

  return X_padded

In [9]:
# Función para realizar la operación de convolución

def convolve(X, F, b):
  """
  Argumentos:
    X: Array numpy de entrada con dimensiones (filter_size, filter_size, n_C_prev)
    F: Array numpy con los pesos de un filtro con dimensiones (filter_size, filter_size, n_C_prev)
    b: Entero con el valor de bias de la capa actual

    IMPORTANTE: EN ESTE CASO, TANTO EL FILTRO COMO EL INPUT TIENEN LA MISMA DIMENSIÓN 
  Retorna:
    Z: Entero con el valor del resultado
  """

  # Multiplico elemento a elemento el valor de entrada con los pesos del filtro
  aux = X * F
  # Realizo la suma de todos los elementos
  aux = np.sum(aux)
  # Le sumo el valor del bias para obtener Z
  Z = aux + float(b)

  return Z

In [12]:
# Función para realizar el forward pass de una capa convolucional

def conv_forward(layer_input, F, b, stride, padding_mode):
  """
  Argumentos:
    layer_input: Array numpy con los valores de entrada a la capa convolucional (batch_size, n_C_prev, n_H_prev, n_W_prev)
    F: Array numpy con los pesos de los filtros utilizados en la capa actual (n_C, n_C_prev, filter_size, filter_size)
    b: Array numpy con los valores de bias utilizados en la capa actual (1, 1, 1, n_C)
    stride: Entero con el valor de stride utilizado en la capa actual.
    padding: 
    padding_mode: {'valid' , 'same'}. Valid = no padding. same --> output.shape = input.shape

  Retorna:
    Z: Array numpy con los valores de salida de la capa convolucional (batch_size, n_H, n_W, n_C)
  """

  # Obtengo las dimensiones de la entrada
  (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape

  # Obtengo las dimensiones de los filtros
  (n_C, n_C_prev, filter_size, filter_size) = W.shape

  # Calculo las dimensiones del volumen de salida de la capa actual
  pad_h = np.divide((F.shape[1]-int(1)),2)
  pad_w = np.divide((F.shape[2]-int(1)),2)
  n_H = int((n_H_prev + 2*pad_h - filter_size)/stride + 1)  ## ((n+2p-f)/s)+1
  n_W = int((n_W_prev + 2*pad_w - filter_size)/stride + 1)  ## ((n+2p-f)/s)+1

  # Inicializo el volumen de salida con ceros
  Z = np.zeros([batch_size, n_C, n_H, n_W])

  # Agrego padding con ceros al volumen de entrada
  layer_input_padded = zero_padding(layer_input, padding_mode, F)

  # Comienzo iterando sobre cada ejemplo del batch
  for i in range(batch_size):

    # Itero sobre el eje vertical del volumen de salida
    for h in range(n_H):
      # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
      y_start = stride * h
      y_end = y_start + filter_size

      # Itero sobre el eje horizontal del volumen de salida
      for w in range(n_W):
        # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        x_start = stride * w
        x_end = x_start + filter_size

        # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
        slice_from_input_padded = layer_input_padded[i, :, y_start:y_end, x_start:x_end]
        
        # Itero sobre la cantidad de canales del volumen de salida
        for c in range(n_C):

          # Obtengo el valor del filtro y bias del canal correspondiente
          filter = W[c, :, :, :]
          bias = b[c]

          # Computo la operación de convolución para esta ventana
          Z[i, c, h, w] = convolve(slice_from_input_padded, filter, bias)
  
  return Z

In [15]:
# Dimensiones de la entrada
batch_size = 10
input_height, input_width = (7, 7)
input_channels = 4

# Dimensiones de la convolucional
filters = 8
filter_size = 3
stride = 2
padding_mode = 'same'

# Variables de prueba
test_array = np.random.randn(batch_size, input_channels, input_height, input_width)
W = np.random.randn(filters, input_channels, filter_size, filter_size)
b = np.random.randn(filters)


conv_result = conv_forward(test_array, W, b, stride, padding_mode)
conv_result_pyt = torch.nn.functional.conv2d(torch.tensor(test_array), torch.tensor(W), torch.tensor(b), stride, pad)

assert(conv_result.shape == conv_result_pyt.shape)
print("Convolución: Result shape: {}".format(conv_result.shape))
print("Convolución: Result value: {}".format(conv_result[1, 1, 1, 1]))
print("Convolución: Result shape: {}".format(conv_result_pyt.shape))
print("Convolución: Result value: {}".format(conv_result_pyt[1, 1, 1, 1]))

Convolución: Result shape: (10, 8, 4, 4)
Convolución: Result value: -0.21671745339529735
Convolución: Result shape: torch.Size([10, 8, 4, 4])
Convolución: Result value: -0.2167174533952978


Función de costo

In [20]:
def loss(conv_result, y):
    """
    Arguments:
    conv_result = resultado del método Foward
    y = Valor verdadero

    Retorna:
    mse = error cuadrático medio

    """
    if (conv_result.size==y.size):
        difference_array = np.subtract(conv_result, y)
        squared_array = np.square(difference_array)
        mse = squared_array.mean()
    else:
        print('La matriz de predicción no tiene la misma dimensión que la matriz salida real')
    return mse

Método Backward

In [21]:
def derivada_loss(conv_result, y):
    """
    Arguments:
    conv_result = resultado del método Foward
    y = Valor verdadero

    Retorna:
    derivada mse = derivada del error cuadrático medio
    """
    difference_array = np.subtract(conv_result, y)
    deriv_mse = np.multiply(-2,difference_array.mean())
    return deriv_mse

In [ ]:
# Función para realizar el backward de una capa convolucional

def conv_backward(layer_input, deriv):
  """
  Argumentos:
    layer_input: Array numpy con los valores de entrada a la capa convolucional (batch_size, n_C, n_H, n_W)
    deriv: Array numpy con las derivadas de la función de costo (n_C, n_C_prev, deriv_size_h, deriv_size_w)

  Retorna:
    deltaL_delta_F: Array numpy con los valores del error respecto al filtro F (batch_size, n_h, n_W, n_C)
  """

  # Obtengo las dimensiones de la entrada
  (batch_size, n_C, n_H, n_W) = layer_input.shape

  # Obtengo las dimensiones de deriv
  (n_C, n_C_prev, deriv_size_h, deriv_size_w) = deriv.shape

  # Calculo las dimensiones del volumen de salida de la capa actual
  #pad_h = np.divide((F.shape[1]-int(1)),2)
  #pad_w = np.divide((F.shape[2]-int(1)),2)
  #n_H = int((n_H_prev + 2*pad_h - filter_size)/stride + 1)  ## ((n+2p-f)/s)+1
  #n_W = int((n_W_prev + 2*pad_w - filter_size)/stride + 1)  ## ((n+2p-f)/s)+1

  # Inicializo el volumen de salida con ceros
  backward = np.zeros([batch_size, n_C, n_H, n_W])

  # Agrego padding con ceros al volumen de entrada
  #layer_input_padded = zero_padding(layer_input, padding_mode, F)

  # Comienzo iterando sobre cada ejemplo del batch
  for i in range(batch_size):

    # Itero sobre el eje vertical del volumen de salida
    for h in range(n_H):
      # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
      y_start = h
      y_end = y_start + deriv_size_h

      # Itero sobre el eje horizontal del volumen de salida
      for w in range(n_W):
        # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        x_start = w
        x_end = x_start + deriv_size_w

        # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
        slice_from_input = layer_input[i, :, y_start:y_end, x_start:x_end]
        
        # Itero sobre la cantidad de canales del volumen de salida
        for c in range(n_C_prev):

          # Obtengo el valor del filtro y bias del canal correspondiente
          filter = deriv[c, :, :, :]

          # Computo la operación de convolución para esta ventana
          backward[i, c, h, w] = conv_backward(slice_from_input, deriv)
  
  return backward